In [1]:
from setup_general import *
from setup_embedding import *

# remove unneeded cols

In [ ]:
data = curie.copy()
data.drop(columns=['text_features'], inplace=True)
data.set_index('id', inplace=True)
split = pd.DataFrame(data.curie_similarity.tolist(), index=data.index)
concat = pd.concat([data, split], axis=1)
concat.drop(columns=['curie_similarity'], inplace=True)
concat.to_csv('data/text_embeddings/curie.csv')
curie = pd.read_csv('data/text_embeddings/curie.csv', index_col='id', dtype={'type': str})
train = curie[curie.source == 'train']
test = curie[curie.source == 'test']
train.drop(columns=['source'], inplace=True)
test.drop(columns=['source'], inplace=True)
train.to_csv('data/text_embeddings/train_curie.csv')
test.to_csv('data/text_embeddings/test_curie.csv')

# train val split


In [64]:
data = train_prep.copy()

features = data.drop('type', axis=1)
labels = data.type
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.3, random_state=0)

In [20]:
X_train.head()

,ks,musealia_seria_nr,musealia_queue_nr,collection_queue_nr,element_count,material_Polish,material_RC Photo Paper,material_acetate cellulose film,material_albumen paper,material_albumin paper,material_aluminium,material_artificial material,material_atlas,material_bone,material_brass,material_bronze,material_canvas,material_canvas (type of cloth),material_cardboard,material_celluloid,material_ceramics,material_charcoal,material_chromogen emulsion,material_chromogen paper,material_clay,material_colloid paper,material_colour,material_copper,material_cotton,material_crystal,material_email,material_emulsion,material_feather,material_film,material_film (material),material_flint,material_glass,material_gold,material_granite,material_graphite,material_gypsum,material_handmade paper,material_ink,material_iron,material_linen,material_mascara,material_metal,material_moire (clothing variety),material_movie,material_nan,material_newsprint,material_nitrocellulose film,material_oil paint,material_organic matter,material_paper,material_photo emulsion,material_photo material,material_photo paper,material_photo plate,material_photographic material,material_plastic,material_plastic mass,material_plywood,material_porcelain,material_printing ink,material_quartz,material_rubber,material_salt paper,material_silk,material_silver,material_silver gelatin emulsion,material_silver gelatin paper,material_skin,material_stone,material_synthetic fibers,material_synthetic material,material_textile,material_tin,material_trillion,material_watercolor paint,material_wax,material_white metal,material_wood,material_wood material,technique_(close/together) sewing,technique_addition,technique_aquatint,technique_ballpoint pen,technique_binding techniques,technique_black and white photo,technique_black and white photography,technique_carving,technique_casting,technique_chalk,technique_charcoal,technique_chromogen procedure,technique_collotype,technique_color photo,technique_color photography,technique_colored chalk,technique_colored pencil,technique_coloring,technique_copper engraving,technique_copying,technique_crocheting,technique_cuttlefish,technique_digital photography,technique_drawing,technique_electronic imaging,technique_embroidery,technique_enamelling,technique_engraving,technique_etching,technique_felt tip pen,technique_glass technology,technique_gouache,technique_graphite,technique_handicraft,technique_handwriting,technique_ink,technique_ink drawing,technique_kiln ceramics,technique_letterpress with raster cliché,technique_linocut,technique_lithography,technique_manuscript,technique_mascara,technique_mezzotinto,technique_mixed media,technique_modeling,technique_molding,technique_monotypy,technique_nan,technique_oil,technique_painting,technique_painting techniques,technique_pannotype,technique_pastel,technique_pen,technique_photographic techniques,technique_photography,technique_photogravure,technique_photolithography,technique_photomechanical printing,technique_polishing,technique_pressing,technique_printing,technique_raster printing,technique_relief,technique_sewing,technique_stamping,technique_steel engraving,technique_taking pictures,technique_tempera,technique_toning,technique_turning,technique_typescript,technique_typing,technique_watercolor,technique_wet collodion process,technique_wood engraving,technique_woodcut,technique_writing,* IN *,amount IN pieces,circumference IN cm,diameter IN cm,distance IN cm,document volume IN pages,height IN cm,image height IN cm,image width IN cm,length IN cm,page height IN cm,page width IN cm,paper format IN A6,print volume IN pages,size (clothing) IN number,thickness IN cm,time/ duration IN century,volume of the book IN pages,volume of wood IN dm3,volume of writing IN autograph,weight IN g,width IN cm,film frame IN mm²_height,film frame IN mm²_width,negative format IN mm²_height,negative format IN mm²_width,photo format IN mm²_height,photo format IN mm²_width,startYear,startMonth,startDay,endYear,endMonth,endDay,locati

In [59]:
tr_curie = pd.DataFrame.join(X_train[['ks']], train_curie)

In [60]:
tr_curie.dropna(axis=0, inplace=True)

In [61]:
len(tr_curie)

7284

In [62]:
tr_curie.head()

ks            type         0         1         2         3  \
id                                                                         
2077459   3486.0  photo negative  0.001483  0.018729  0.014191  0.025457   
3919154   1063.0          poster  0.004305  0.015591  0.006037  0.031716   
3782485     93.0     music sheet  0.022531  0.004523  0.023973  0.026333   
2981166    253.0           photo  0.007618  0.025631  0.003756  0.002841   
3632522  35838.0  photo negative -0.007513  0.019764  0.006324  0.021602   

                4         5         6         7         8         9        10  \
id                                                                              
2077459 -0.007948  0.013446 -0.012011 -0.003997  0.020294  0.027097 -0.033116   
3919154  0.001927  0.014532 -0.010600 -0.008942  0.009476  0.020665 -0.014486   
3782485  0.001547  0.007122 -0.001619  0.005623  0.024909  0.020901 -0.025396   
2981166  0.001137  0.015688 -0.003845 -0.001633  0.009655  0.014622 -0.051532   
3632522 -0.015737  0.015017 -0.006168 -0.009323 -0.000159  0.014809 -0.026396   

               11        12        13        14        15        16        17  \
id                                                                              
2077459 -0.019214 -0.016660  0.008223  0.003599 -0.011200  0.003280 -0.012020   
3919154 -0.009310 -0.020315  0.008458 -0.004954 -0.019652 -0.001796 -0.001921   
3782485 -0.014262  0.000736  0.016959  0.006480 -0.022924  0.001607 -0.016453   
2981166 -0.000788 -0.002928  0.024017  0.000725 -0.021058  0.007585 -0.013584   
3632522 -0.015254  0.000280  0.008811  0.012687 -0.015510  0.002696 -0.015302   

               18        19        20        21        22        23        24  \
id                                                                              
2077459  0.007133  0.003380 -0.005558  0.008447 -0.012225  0.012505  0.006061   
3919154 -0.005797  0.006018 -0.007280  0.001033 -0.013123 -0.014136  0.018529   
3782485  0.021782  0.003240 -0.000147 -0.001477 -0.021894  0.008241  0.000541   
2981166  0.013094  0.012672 -0.000302  0.000447 -0.016620 -0.008776  0.022807   
3632522 -0.008617  0.005870 -0.012336  0.002745 -0.011076 -0.002845  0.034962   

               25        26        27        28        29        30        31  \
id                                                                              
2077459  0.015775  0.002516 -0.037160 -0.010371  0.029650  0.008218 -0.003275   
3919154  0.010489  0.006248 -0.026356 -0.010130  0.024128 -0.007869 -0.011613   
3782485  0.006649 -0.012314 -0.021763 -0.013635  0.019422 -0.000735  0.003992   
2981166  0.009405 -0.000314 -0.034816 -0.013997  0.021251  0.003310 -0.007148   
3632522  0.010299  0.000876 -0.031532 -0.009451  0.018030 -0.002171 -0.016069   

               32        33        34        35        36        37        38  \
id                                                                              
2077459  0.014555  0.077749  0.010986  0.013539  0.012551 -0.007384 -0.001901   
3919154  0.014385  0.090838 -0.000751  0.017433  0.006184 -0.009992  0.014863   
3782485  0.002966  0.089899  0.017867  0.006564  0.018148 -0.000311  0.003397   
2981166  0.006398  0.069362 -0.014958  0.007810  0.004270 -0.002337  0.011970   
3632522  0.010233  0.093913  0.015036  0.017405  0.007419  0.014127 -0.000351   

               39        40        41        42        43        44        45  \
id                                                                              
2077459 -0.121282 -0.003322  0.004813 -0.005991 -0.000397  0.026668  0.004943   
3919154 -0.114781 -0.002162  0.006386 -0.008799 -0.007846  0.020260  0.007841   
3782485 -0.106305 -0.007037  0.013063  0.007398 -0.005558  0.010366  0.002793   
2981166 -0.092534 -0.007863  0.031684 -0.004904 -0.011961  0.016726  0.002271   
3632522 -0.120215 -0.004865  0.018798 -0.002012 -0.014269  0.022720  0.012601   

               46        47        48        49        50        51      

In [65]:
val_curie = pd.DataFrame.join(X_val[['ks']], train_curie)

In [66]:
val_curie.dropna(axis=0, inplace=True)

In [67]:
len(val_curie)

3166

In [68]:
val_curie.head()

ks                   type         0         1         2  \
id                                                                      
2080460  19335.0         photo negative -0.004393  0.002888  0.012602   
3195806     34.0  design/drawing/sketch  0.000258  0.006448  0.010911   
3671158      0.0          digital image -0.005665  0.016561  0.018534   
4092912    575.0                 poster  0.016871  0.023630  0.008212   
3173802     15.0                   plan  0.000349  0.010065 -0.007989   

                3         4         5         6         7         8         9  \
id                                                                              
2080460  0.023582 -0.015369  0.019766 -0.007274 -0.005662  0.013499  0.027150   
3195806  0.033136 -0.000063  0.008050 -0.003758  0.006579  0.017598  0.021410   
3671158  0.019405 -0.011954  0.006120 -0.012854  0.004126  0.007868  0.025516   
4092912  0.019683 -0.015605  0.013147 -0.024357 -0.006690  0.001335  0.022718   
3173802  0.023998 -0.020148  0.027445  0.000418 -0.006045  0.007975  0.025483   

               10        11        12        13        14        15        16  \
id                                                                              
2080460 -0.029115 -0.006773  0.000983  0.014214  0.004350 -0.020530 -0.012154   
3195806 -0.008406 -0.000836 -0.005114  0.016006  0.000736 -0.014077  0.000208   
3671158 -0.022585 -0.004720 -0.018869  0.021072  0.014559 -0.024501 -0.010718   
4092912 -0.018826 -0.008151 -0.004660  0.014692  0.008994 -0.024357 -0.001997   
3173802 -0.016161 -0.011697 -0.005706  0.023008  0.011413 -0.006540 -0.003761   

               17        18        19        20        21        22        23  \
id                                                                              
2080460 -0.012946  0.005371  0.008199  0.002521 -0.012497 -0.007460  0.004443   
3195806 -0.013590 -0.000866  0.009122 -0.015753  0.005797 -0.007839 -0.008846   
3671158 -0.002183  0.009837  0.003829 -0.004008 -0.009592 -0.011886  0.005771   
4092912 -0.010512  0.018267  0.003889 -0.011061 -0.014338 -0.004432  0.009674   
3173802 -0.021157  0.007411  0.010423 -0.006325 -0.015840 -0.008158  0.010019   

               24        25        26        27        28        29        30  \
id                                                                              
2080460  0.020110 -0.003594 -0.010465 -0.033504  0.003470  0.016571 -0.002499   
3195806  0.016062  0.002494 -0.002084 -0.016709 -0.001027  0.023752  0.008551   
3671158  0.031838  0.002885 -0.010287 -0.018333 -0.004387  0.009219 -0.010967   
4092912  0.007090  0.003459  0.004297 -0.032941 -0.007146  0.014636  0.002002   
3173802  0.017508  0.015345 -0.006577 -0.046457 -0.013429  0.023302 -0.022862   

               31        32        33        34        35        36        37  \
id                                                                              
2080460  0.000782  0.006811  0.095474 -0.007613  0.013919  0.006520  0.002019   
3195806  0.004357  0.014882  0.123554  0.014639  0.013730  0.010593  0.004215   
3671158 -0.015555  0.015430  0.096777  0.003951  0.009856  0.017854 -0.007538   
4092912  0.001860  0.016657  0.098022 -0.001613  0.019683  0.004581  0.003005   
3173802 -0.014282  0.011009  0.108754  0.012531  0.016693  0.007980  0.000126   

               38        39        40        41        42        43        44  \
id                                                                              
2080460  0.006220 -0.115164 -0.020663  0.011047  0.007408 -0.015979  0.018126   
3195806  0.002768 -0.127000  0.010818 -0.004343 -0.006781 -0.007099  0.029184   
3671158  0.008994 -0.104440 -0.013218  0.016589 -0.002541 -0.008606  0.020899   
4092912  0.009464 -0.118058 -0.019776  0.020334 -0.005247 -0.008170  0.012039   
3173802  0.000230 -0.102007 -0.007320  0.008718 -0.000706  0.002388  0.020093   

               45        46        47        48        49        50        51  \
id           

In [72]:
tr_curie.to_csv('data/text_embeddings/train_curie.csv', index_label='id')
val_curie.to_csv('data/text_embeddings/val_curie.csv', index_label='id')

: 